In [ ]:
!pip install ultralytics

In [17]:
from google.colab import drive
import cv2
import os
import math
from ultralytics import YOLO
import matplotlib.pyplot as plt

In [ ]:
drive.mount('/content/drive')

# Extract frames from video

In [20]:
def create_directories(base_path):
    os.makedirs(f"{base_path}/train/moodeng", exist_ok=True)
    os.makedirs(f"{base_path}/train/box", exist_ok=True)
    os.makedirs(f"{base_path}/val/moodeng", exist_ok=True)
    os.makedirs(f"{base_path}/val/box", exist_ok=True)


def find_video_file(folder_path, file_prefix):
    possible_extensions = ['.mp4', '.mov']  
    file_prefix_lower = file_prefix.lower()

    for file in os.listdir(folder_path):
        for ext in possible_extensions:
            if file.lower() == f"{file_prefix_lower}{ext}":
                return os.path.join(folder_path, file)
    return None

def save_frame(image, output_dir, prefix, count):
    file_name = os.path.join(output_dir, f"{count:04d}_{prefix}.png")
    cv2.imwrite(file_name, image)

def split_frames(video_path, base_output_dir, prefix, frames_per_second=5, split_ratio=0.8):
    vidcap = cv2.VideoCapture(video_path)

    if not vidcap.isOpened(): 
        print(f"Error: Could not open video file {video_path}")
        return

    fps = int(vidcap.get(cv2.CAP_PROP_FPS))
    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"Frames per second: {fps}")
    print(f"Total frames in video: {total_frames}")

    frame_interval = int(fps / frames_per_second)

    total_extracted_frames = total_frames // frame_interval

    train_frame_limit = math.ceil(total_extracted_frames * split_ratio)

    count = 0
    success, image = vidcap.read()
    frame_number = 0
    
    while success:
        if frame_number % frame_interval == 0:
            if count < train_frame_limit:
                output_dir = f"{base_output_dir}/train/{prefix}"
            else:
                output_dir = f"{base_output_dir}/val/{prefix}"
            
            save_frame(image, output_dir, prefix, count)
            count += 1

        success, image = vidcap.read()
        frame_number += 1

    vidcap.release()

In [ ]:
def process_single_video(base_path, base_output_dir, file_prefix, frames_per_second=10, split_ratio=0.8):
    video_path = find_video_file(base_path, file_prefix)
    
    if video_path:
        print(f"Found video file: {video_path}")

        create_directories(base_output_dir)

        split_frames(video_path, base_output_dir, file_prefix, frames_per_second, split_ratio)
    else:
        print(f"Error: Video file for {file_prefix} not found.")
base_output_dir = "Data"
base_path = '/content/drive/MyDrive/RAI_CAMP_DATASET' 

process_single_video(base_path, base_output_dir, 'moodeng')
process_single_video(base_path, base_output_dir, 'box')
print("Frame extraction and splitting into train/val completed.")

# Train model

In [ ]:
model = YOLO('yolov8n-cls.pt')
model.train(data='/content/Data', epochs=5)

# Inference

In [23]:
def perform_inference(model, image_path):
    image = cv2.imread(image_path)

    results = model(image_path)

    predicted_class = results[0].names[results[0].probs.top1]
    confidence = results[0].probs.top1conf.item()

    display_image_with_prediction(image, predicted_class, confidence)

def display_image_with_prediction(image, predicted_class, confidence):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(8, 8))
    plt.imshow(image_rgb)
    plt.title(f"Prediction: {predicted_class}, Confidence: {confidence*100:.2f}%")
    plt.axis('off')
    plt.show()

In [ ]:
def process_images_in_folder(model, folder_path):
    supported_extensions = ('.png', '.jpg')

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(supported_extensions):
            image_path = os.path.join(folder_path, filename)
            print(f"Processing image: {image_path}")
            perform_inference(model, image_path)

test_folder_path = '/content/drive/MyDrive/RAI_CAMP_DATASET/TEST'

process_images_in_folder(model, test_folder_path)
